TL;DR: don't read this notebook. use [pandas_market_calendars](https://github.com/rsheftel/pandas_market_calendars?tab=readme-ov-file)

long story:

this notebook was inspired by https://github.com/shinathan/polygon.io-stock-database/blob/main/02_market_hours.ipynb
but AFAIU it is not needed because i found [pandas_market_calendars](https://github.com/rsheftel/pandas_market_calendars?tab=readme-ov-file)

the idea was to download SPY minute bars and assume that since SPY has a lot of volume then it's possible to identify open/close hours for the market by looking at when there is a first trade for SPY.

but the first thing that i noticed was that when i executed the code in https://github.com/shinathan/polygon.io-stock-database/blob/main/02_market_hours.ipynb i got not the same results as the author had. next i checked that the hour of the first trade for SPY is not always 4am as it was assumed by the author.

the result is that AFAIU it's not a good idea to try to identify market hours from SPY.

In [ ]:
%pip install polygon-api-client pandas python-dateutil

In [ ]:
import pandas as pd

# data was loaded by download_bars.py

# Read the CSV file
df = pd.read_csv('../../polygon-data/bars/1minute/SPY.csv')
df

In [ ]:
# get only the timestamp column
timestamps = pd.to_datetime(df['timestamp'])
# truncate hours
hours = timestamps.dt.floor('H')
# remove duplicate hours
unique_hours = hours.drop_duplicates()
unique_hours

In [ ]:
# for each day get the first and last hour
first_hours = unique_hours.groupby(unique_hours.dt.date).first()
last_hours = unique_hours.groupby(unique_hours.dt.date).last()
# create a DataFrame with the first and last hours
first_last_hours = pd.DataFrame({
    'first_hour': first_hours,
    'last_hour': last_hours
})
first_last_hours

In [ ]:
# rows that do not start at 04:00 or 08:00
irregular_opens = first_last_hours[
    (first_last_hours['first_hour'].dt.hour != 4) &
    (first_last_hours['first_hour'].dt.hour != 8)
]
irregular_opens

In [ ]:
%pip install pandas_market_calendars

In [ ]:
import pandas_market_calendars as mcal

# Create a calendar
nyse = mcal.get_calendar('NYSE')

# Show available calendars
# print(mcal.get_calendar_names())
nyse.schedule(start_date='2004-05-20', end_date='2004-05-22')

In [ ]:
irregular_opens

In [ ]:
irregular_closes = first_last_hours[(first_last_hours['last_hour'].dt.hour != 19) &
    (first_last_hours['last_hour'].dt.hour != 20) &
    (first_last_hours['last_hour'].dt.hour != 18)]
irregular_closes

In [ ]:
nyse.schedule(start_date='2024-12-23', end_date='2024-12-30')

one more check - that SPY has at least trades for all days where pandas_market_calendars says that it was a trading day.

In [ ]:
# get days from unique_hours
unique_days = unique_hours.dt.date.drop_duplicates()
first_day = str(unique_days.iloc[0])
last_day = str(unique_days.iloc[-1])
nyse_schedule = nyse.schedule(start_date=first_day, end_date=last_day)
nyse_schedule

In [ ]:
# convert the index to a Series of dates to be comparable with unique_days
nyse_days = pd.Series(nyse_schedule.index.date)

In [ ]:
# check that unique_days are in the NYSE schedule
print(nyse_days.isin(unique_days).all())
# and that the NYSE schedule is in unique_days
print(unique_days.isin(nyse_days).all())

In [ ]:
extended = nyse.schedule(start_date=first_day, end_date=last_day, start="pre", end="post")
extended
